<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analisis-de-sentimiento-en-opiniones-de-IMDB---LogRes-versus-NN" data-toc-modified-id="Analisis-de-sentimiento-en-opiniones-de-IMDB---LogRes-versus-NN-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analisis de sentimiento en opiniones de IMDB - LogRes versus NN</a></span></li><li><span><a href="#1.-Primer-bloque---Carguemos-las-opiniones-para-aplicar-Bag-of-Words" data-toc-modified-id="1.-Primer-bloque---Carguemos-las-opiniones-para-aplicar-Bag-of-Words-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Primer bloque - Carguemos las opiniones para aplicar Bag of Words</a></span></li><li><span><a href="#2.-Primer-bloque---Regresión-Logistica-con-Bag-Of-Words" data-toc-modified-id="2.-Primer-bloque---Regresión-Logistica-con-Bag-Of-Words-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. Primer bloque - Regresión Logistica con Bag Of Words</a></span><ul class="toc-item"><li><span><a href="#2.1-Cargamos-las-librerias-para-la-Regresión-Logística-y-la-entrenamos-con-cross-validation-buscando-el-mejor-parámetro-C-de-la-regresion-logística" data-toc-modified-id="2.1-Cargamos-las-librerias-para-la-Regresión-Logística-y-la-entrenamos-con-cross-validation-buscando-el-mejor-parámetro-C-de-la-regresion-logística-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>2.1 Cargamos las librerias para la Regresión Logística y la entrenamos con cross validation buscando el mejor parámetro C de la regresion logística</a></span></li></ul></li><li><span><a href="#3.Segundo-bloque---Vayamos-a-por-una-red-neuronal" data-toc-modified-id="3.Segundo-bloque---Vayamos-a-por-una-red-neuronal-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3.Segundo bloque - Vayamos a por una red neuronal</a></span></li></ul></div>

## Analisis de sentimiento en opiniones de IMDB - LogRes versus NN

## 1. Primer bloque - Carguemos las opiniones para aplicar Bag of Words

Necesitaremos pandas y el CountVectorizer de sklearn para aplicar bag of words sobre las frases.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import time
import datetime

A cargar datos del archivo imdb_labelled.txt y mostramos su primer valor

In [ ]:
df = pd.read_csv('imdb_labelled.txt', names=['frases','label'], sep='\t')
print(df.iloc[0])

Una valoración bastante negativa... como podeis ver el label 0 es "negativo", así que 1 será "positivo" (analizando el sentimiento de la frase)

Preparemos los datos, para tener un test y un train

In [ ]:
frases = df['frases'].values
y = df['label'].values

In [ ]:
frases_train, frases_test, y_train, y_test = train_test_split(frases, y, test_size=0.25, random_state=1000)

Ahora viene el momento de crear nuestro bag of words, para eso usaremos CountVectorizer de sklearn, que lo entrenaremos con las palabras/frases del train y luego aplicaremos a cada set de frases train y test

In [ ]:
vectorizador = CountVectorizer()
vectorizador.fit(frases_train)

x_train = vectorizador.transform(frases_train)
x_test = vectorizador.transform(frases_test)

In [ ]:
print(x_train[0,:])
print(vectorizador.vocabulary_)

## 2. Primer bloque - Regresión Logistica con Bag Of Words

Para empezar carguemos esas librerías que nos hacen falta

### 2.1 Cargamos las librerias para la Regresión Logística y la entrenamos con cross validation buscando el mejor parámetro C de la regresion logística

In [ ]:
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(x_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)


In [ ]:
lr = grid.best_estimator_
lr.fit(x_train, y_train)
lr.predict(x_test)

score_test = lr.score(x_test, y_test)

print('Accuracy for IMDB data in test data: {:.4f}'.format(score_test))


## 3.Segundo bloque - Vayamos a por una red neuronal

Una red sencilla, por ahora nada de RNN

Vamos a por las librerias que necesitamos

In [ ]:
from tensorflow import keras as ks
from matplotlib import pyplot as plt

Veamos cuantas dimensiones tengo para la entrada de la primera capa de la red neuronal

In [ ]:
input_dim = x_train.shape[1]
print('Numero dimensiones:', input_dim)

Definimos el modelo de red neuronal

In [ ]:
model = ks.Sequential()
model.add(ks.layers.Dense(512, input_dim=input_dim, activation='relu'))
model.add(ks.layers.Dropout(0.5))
model.add(ks.layers.Dense(256, activation='relu'))
model.add(ks.layers.Dropout(0.5))
model.add(ks.layers.Dense(50, activation='relu'))
model.add(ks.layers.Dropout(0.5))
model.add(ks.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', \
              optimizer='adam', \
               metrics=['accuracy'])

Entrenemos el modelo

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), batch_size=96)

In [ ]:
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.show()

plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.show()

Podemos ver que la red neurona facilmente entra en overfitting... asi que la regresión logística, con menos esfuerzo, funcionan parecido.

In [ ]:
_,score_test = model.evaluate(x_test, y_test)

print('Accuracy for IMDB data in test data with NN: {:.4f}'.format(score_test))